In [1]:
import geolib.geohash
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
from tqdm.auto import tqdm
from cri98tj.partitioners.Geohash_partitioner import Geohash_partitioner
from cri98tj.normalizers.FirstPoint_normalizer import FirstPoint_normalizer
from cri98tj.selectors.RandomInformationGain_selector import RandomInformationGain_selector
from cri98tj.distancers.Euclidean_distancer import Euclidean_distancer
from sklearn.model_selection import train_test_split
from cri98tj.distancers.Euclidean_distancer import euclideanBestFitting

In [3]:
df_original = pd.read_csv('../../examples/Animals Dataset/data/animals_preapred.zip').sort_values(by=["tid", "t"])[["tid", "class", "c1", "c2", "t"]]

df_original.head()

,tid,class,c1,c2,t
0,1,D,50.1066,3.79665,0
1,1,D,50.1045,3.79455,4
2,1,D,50.1111,3.79845,7
3,1,D,50.1072,3.79845,9
4,1,D,50.1132,3.79965,15


In [4]:
tid_train, tid_test, _, _ = train_test_split(df_original.groupby(by=["tid"]).max().reset_index()["tid"],
                                                        df_original.groupby(by=["tid"]).max().reset_index()["class"],
                                                        test_size=.3,
                                                        stratify=df_original.groupby(by=["tid"]).max().reset_index()["class"],
                                                        random_state=3)

df = df_original.copy()

In [5]:
from datetime import datetime

df_train = df_original[df_original.tid.isin(tid_train)].copy()

df_train["date"] = df_train.t.apply(lambda x: datetime.fromtimestamp(x).strftime("%Y-%m-%d"))
df_train.t = df_train.t.apply(lambda x: datetime.fromtimestamp(x).strftime("%H:%M:%S"))
df_train.c1 = df_train.c1.apply(lambda x: x*(10**6))
df_train.c2 = df_train.c2.apply(lambda x: x*(10**6))

#campi fittizzi
df_train["signal"] = 0
df_train["heading"] = 0
df_train["quality"] = 3
df_train["status"] = 2
df_train["delta"] = 0


df_train = df_train[["tid", "date", "t", "c1", "c2", "signal", "heading", "quality", "status", "delta"]]

df_train

,tid,date,t,c1,c2,signal,heading,quality,status,delta
0,1,1970-01-01,01:00:00,50106600.0,3796650.0,0,0,3,2,0
1,1,1970-01-01,01:00:04,50104500.0,3794550.0,0,0,3,2,0
2,1,1970-01-01,01:00:07,50111100.0,3798450.0,0,0,3,2,0
3,1,1970-01-01,01:00:09,50107200.0,3798450.0,0,0,3,2,0
4,1,1970-01-01,01:00:15,50113200.0,3799650.0,0,0,3,2,0
...,...,...,...,...,...,...,...,...,...,...
14863,101,1970-01-01,01:04:20,50176800.0,3761850.0,0,0,3,2,0
14864,101,1970-01-01,01:04:22,50176500.0,3760950.0,0,0,3,2,0
14865,101,1970-01-01,01:04:24,50175300.0,3760050.0,0,0,3,2,0
14866,101,1970-01-01,01:04:26,50171700.0,3765450.0,0,0,3,2,0


In [6]:
df_train.to_csv("gps_.csv", index=None, header=None)

In [7]:
!cat gps_.csv | sort -t',' -k1 -k2 -k3 -k9 -m | gzip -c > gps_sorted.csv.gz

In [12]:
df

,tid,class,c1,c2,t
0,1,D,50.1066,3.79665,0
1,1,D,50.1045,3.79455,4
2,1,D,50.1111,3.79845,7
3,1,D,50.1072,3.79845,9
4,1,D,50.1132,3.79965,15
...,...,...,...,...,...
14985,102,C,50.1546,3.76005,243
14986,102,C,50.1540,3.75975,245
14987,102,C,50.1546,3.76065,247
14988,102,C,50.1573,3.75855,249


In [8]:
import skmob
from skmob.preprocessing import filtering, compression
import geopandas as gpd

In [23]:
tdf = skmob.TrajDataFrame(df.rename(columns={"c1": "lat", "c2": "lng"}))
tessellation = gpd.read_file("voronoi.shp").reset_index().rename(columns={"index": "tile_ID"})

In [24]:
map_tdf = tdf.mapping(tessellation, remove_na=True)

/home/cri98li/Documenti/GitHub/cri98tj/venv/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/home/cri98li/Documenti/GitHub/cri98tj/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/cri98li/Documenti/GitHub/cri98tj/venv/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:322: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to matc

In [25]:
map_tdf

,tid,class,lat,lng,t,tile_ID
0,1,D,50.1066,3.79665,0,132
1,1,D,50.1045,3.79455,4,132
2,1,D,50.1111,3.79845,7,143
3,1,D,50.1072,3.79845,9,132
4,1,D,50.1132,3.79965,15,143
...,...,...,...,...,...,...
14985,102,C,50.1546,3.76005,243,51
14986,102,C,50.1540,3.75975,245,51
14987,102,C,50.1546,3.76065,247,51
14988,102,C,50.1573,3.75855,249,47
